# Python 简单的宝具 `高通量` 伤害计算器 (误)

原文链接 [NGA 论坛](https://nga.178.com/read.php?tid=20289131) @惟江上之轻舟与舟上之清粥

代码改自主楼

In [2]:
import pandas as pd
import numpy as np
import ast
import itertools

# 输出 4 位有效数字，每行最多输出 150 字符
np.set_printoptions(4, linewidth=150, suppress=True)

## 数据与全局变量定义

读入 xlsx 文件数据：

In [2]:
df_support = pd.read_excel('队友.xlsx', encoding='gbk')
df_craft_essence = pd.read_excel('礼装.xlsx', encoding='gbk')
df_mystic_code = pd.read_excel('衣服.xlsx', encoding='gbk')
df_enemy = pd.read_excel('敌人.xlsx', encoding='gbk')
df_noble_all = pd.read_excel('光炮从者.xlsx',encoding = 'gbk')
df_class_relation = pd.read_excel('职阶克制表.xlsx',encoding = 'gbk')

定义职阶克制关系 (根据文件 `职阶克制表.xlsx`)

In [3]:
# servant class relationship
# CLASS_RELATION => row: attacker
#                => col: defender
CLASS_TYPE = np.asarray(df_class_relation.columns, dtype=str)
CLASS_RELATION = {}
for cls, relation in zip(CLASS_TYPE, np.asarray(df_class_relation)):
    CLASS_RELATION[cls] = relation
CLASS_RELATION

{'S': array([1. , 0.5, 2. , 1. , 1. , 1. , 2. , 1. , 0.5, 1. , 1. , 1. , 1. , 1. , 1. , 1. ]),
 'A': array([2. , 1. , 0.5, 1. , 1. , 1. , 2. , 1. , 0.5, 1. , 1. , 1. , 1. , 1. , 1. , 1. ]),
 'L': array([0.5, 2. , 1. , 1. , 1. , 1. , 2. , 1. , 0.5, 1. , 1. , 1. , 1. , 1. , 1. , 1. ]),
 'R': array([1. , 1. , 1. , 1. , 2. , 0.5, 2. , 1. , 0.5, 1. , 1. , 1. , 1. , 2. , 1. , 1. ]),
 'C': array([1. , 1. , 1. , 0.5, 1. , 2. , 2. , 1. , 0.5, 1. , 1. , 1. , 1. , 2. , 1. , 1. ]),
 'AS': array([1. , 1. , 1. , 2. , 0.5, 1. , 2. , 1. , 0.5, 1. , 1. , 1. , 1. , 2. , 1. , 1. ]),
 'B': array([1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1. , 1.5, 1.5, 1.5, 1.5, 0.5, 1.5, 1. , 1. ]),
 'SH': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'RU': array([1. , 1. , 1. , 1. , 1. , 1. , 2. , 1. , 1. , 0.5, 2. , 1. , 1. , 1. , 1. , 1. ]),
 'AV': array([1. , 1. , 1. , 1. , 1. , 1. , 2. , 1. , 2. , 1. , 0.5, 1. , 1. , 1. , 1. , 1. ]),
 'MC': array([1. , 1. , 1. , 1. , 1. , 1. , 2. , 1. , 0.5, 2. 

定义天地人属性关系

In [4]:
# servant attribute relationship
# ATTRIBUTE_RELATION => row: attacker
#                    => col: defender
ATTRIBUTE_TYPE = np.array(['tian', 'di', 'ren', 'xing', 'shou'])
tmp = np.array([
    [1.0, 1.1, 0.9, 1.0, 1.0],
    [0.9, 1.0, 1.1, 1.0, 1.0],
    [1.1, 0.9, 1.0, 1.0, 1.0],
    [1.0, 1.0, 1.0, 1.0, 1.1],
    [1.0, 1.0, 1.0, 1.1, 1.0],
])
ATTRIBUTE_RELATION = {}
for attr, relation in zip(ATTRIBUTE_TYPE, tmp):
    ATTRIBUTE_RELATION[attr] = relation
ATTRIBUTE_RELATION

{'tian': array([1. , 1.1, 0.9, 1. , 1. ]),
 'di': array([0.9, 1. , 1.1, 1. , 1. ]),
 'ren': array([1.1, 0.9, 1. , 1. , 1. ]),
 'xing': array([1. , 1. , 1. , 1. , 1.1]),
 'shou': array([1. , 1. , 1. , 1.1, 1. ])}

定义从者与敌方属性 (根据文件 `光炮从者.xlsx` 与 `敌人.xlsx`)

In [5]:
# trait list
# https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
TRAIT_TYPE = np.unique(
    list(itertools.chain.from_iterable([i.split("，") for i in df_enemy["特性"]])) + \
    list(itertools.chain.from_iterable([ast.literal_eval(i).keys() for i in df_noble_all['特攻'][df_noble_all['特攻'] != 0]])) + \
    list(itertools.chain.from_iterable([ast.literal_eval(i).keys() for i in df_noble_all['宝具特攻倍率'][df_noble_all['宝具特攻倍率'] != 1]]))
)
TRAIT_ONEHOT = {}
for trait, onehot in zip(TRAIT_TYPE, np.eye(TRAIT_TYPE.size)):
    TRAIT_ONEHOT[trait] = onehot
TRAIT_TYPE

array(['人型', '从者', '天地从者', '恶', '猛兽', '男性', '神性', '骑乘', '魔性', '龙'], dtype='<U4')

定义色卡的 one-hot 变量

In [6]:
# card color definition
COLOR_ONEHOT = {
    1: np.array([1, 0, 0]),
    2: np.array([0, 1, 0]),
    3: np.array([0, 0, 1]),
}

## 计算程序

结果应当是用四维的特征表示：

- 从者 (Servant)
- 友军 (Support)
- 礼装 (Craft Essence)
- 御主服装 (Mystic Code)

为了计算从者的输出，我们除了需要上述的数据之外，还需要敌方的数据。我们先建立敌方变量所属的类：

In [7]:
class Enemy:
    def __init__(self, enemy_dict):
        self.cls = enemy_dict["职阶"]
        self.attribute = enemy_dict["阵营"]
        self.trait = self.str_to_trait(enemy_dict["特性"])
    
    @staticmethod
    def str_to_trait(s):
        trait_array = np.zeros(TRAIT_TYPE.size)
        for trait in s.split("，"):
            trait_array += TRAIT_ONEHOT[trait]
        return trait_array

譬如以第三个敌人为例：

- `cls` 敌方职阶，由于 `class` 在 python 中是保护词，因此取用单词缩写
- `attribute` 敌方属性 (天地人)
- `trait` 敌方特性，用 01 编码来表示 (但为了程序便利，没有采用布尔值)

In [8]:
enemy = Enemy(df_enemy.iloc[2])
print(enemy.cls)
print(enemy.attribute)
print(enemy.trait)

S
di
[1. 0. 0. 0. 1. 1. 0. 0. 0. 0.]


其等价的对应的信息是

In [9]:
df_enemy.iloc[2]

敌人          雅嘎
职阶           S
阵营          di
特性    男性，人型，猛兽
Name: 2, dtype: object

下面是一个相当大的类 `Noble_All`，用以储存从者、友军、礼装、御主服装的信息。其中，

- `generate_trait_extra`
    - 用于生成从者特攻和宝具特攻倍率的信息，不需要在外部使用
- `calculate_damage`
    - 输入变量 `enemy: Enemy`，一次性计算所有情况下对某敌方的伤害
    - 输出为 4-dimension ndarray dtype=float32，维度信息是 (从者, 友军, 礼装, 御主服装)
- `calculate_np` 一次性计算所有情况下从者所能达到的最大宝具值
    - 输出为 4-dimension ndarray dtype=float32，维度信息是 (从者, 友军, 礼装, 御主服装)
- `output`
    - 给出满足一次性能充满 100% 宝具值，以及伤害大于 100,000 的宝具攻击情况
    - 输入变量 `enemy: Enemy`，以及可选变量 `slic` 表示截取数据的范围 (但仍然计算所有可能的宝具伤害情况)
    - 输出 pandas dataframe，并写入外部 xlsx 文件

In [10]:
class Noble_All:
    
    def __init__(self):
        self.df_support = df_support
        self.df_craft_essence = df_craft_essence
        self.df_mystic_code = df_mystic_code
        self.df_enemy = df_enemy
        self.df_noble_all = df_noble_all
        self.df_class_relation = df_class_relation
    
        self.servant_name = np.asarray(self.df_noble_all["从者"])
        self.support_name = np.asarray(self.df_support["组合代号"])
        self.craft_name = np.asarray(self.df_craft_essence["礼装代号"])
        self.mystic_name = np.asarray(self.df_mystic_code["衣服代号"])
    
        self.servant_atk = np.asarray(self.df_noble_all["ATK"])
        self.servant_base_ratio = np.asarray(self.df_noble_all["宝具倍率"])
        self.servant_color = np.array([COLOR_ONEHOT[color] for color in self.df_noble_all["宝具颜色"]])
        self.servant_card_ratio = np.asarray(self.df_noble_all["色卡倍率"])
        self.servant_color_powerup = np.asarray(self.df_noble_all["色卡性能"])
        self.servant_class_multiplier = np.asarray(self.df_noble_all["职阶"])
        self.servant_class_relation = np.array([CLASS_RELATION[cls] for cls in self.df_noble_all["职阶倍率"]])
        self.servant_attribute_relation = np.array([ATTRIBUTE_RELATION[attr] for attr in self.df_noble_all["阵营"]])
        self.servant_atk_powerup = np.asarray(self.df_noble_all["攻防"])
        self.servant_skill_extra = self.generate_trait_extra("特攻", 0)          # base value is 0
        self.servant_noble_powerup = np.asarray(self.df_noble_all["宝具buff"])
        self.servant_noble_extra = self.generate_trait_extra("宝具特攻倍率", 1)  # base value is 1
        self.servant_np = np.asarray(self.df_noble_all["np"])
        
        self.support_atk_powerup = np.asarray(self.df_support["攻击力提升"])
        self.support_color_powerup = np.asarray(self.df_support[[1, 2, 3]])
        self.support_noble_powerup = np.asarray(self.df_support["宝具威力提升"])
        self.support_atk_addition = np.asarray(self.df_support["伤害附加"])
        self.support_np = np.asarray(self.df_support["np"])
        
        self.craft_atk = np.asarray(self.df_craft_essence["ATK"])
        self.craft_atk_powerup = np.asarray(self.df_craft_essence["攻击力提升"])
        self.craft_color_powerup = np.asarray(self.df_craft_essence[[1, 2, 3]])
        self.craft_noble_powerup = np.asarray(self.df_craft_essence["宝具威力提升"])
        self.craft_np = np.asarray(self.df_craft_essence["np"])
        
        self.mystic_atk_powerup = np.asarray(self.df_mystic_code["攻击力提升"])
        self.mystic_color_powerup = np.asarray(self.df_mystic_code[[1, 2, 3]])
        self.mystic_noble_powerup = np.asarray(self.df_mystic_code["宝具威力提升"])
        self.mystic_np = np.asarray(self.df_mystic_code["np"])
    
    def generate_trait_extra(self, key, basevalue):
        servant_trait_extra = []
        for val in self.df_noble_all[key]:
            current_trait_extra = np.zeros(TRAIT_TYPE.size)
            if val is not basevalue:
                for trait, extra in ast.literal_eval(val).items():
                    current_trait_extra += TRAIT_ONEHOT[trait] * (extra - basevalue)
            servant_trait_extra += [current_trait_extra]
        return np.array(servant_trait_extra)
    
    def calculate_damage(self, enemy):
        # 函数编写的原则：其中类似于 [None, :, None, None] 一般都有以下原则，
        #                 从者冒号放在第 0 位，友军第 1 位，礼装第 2 位，御主服装第 3 位
        #                 色卡存在第 4 位表示色卡值，需要用 one-hot 的色卡向量作内积求和
        return (
            0.23   # 本身的倍数
            * 0.9  # 随机情况中最背的情况
            * (self.servant_atk[:, None, None, None] + self.craft_atk[None, None, :, None])  # 基础 ATK 与礼装 ATK 值
            * self.servant_base_ratio[:, None, None, None]  # 宝具倍率
            * self.servant_card_ratio[:, None, None, None]  # 色卡倍率
            * (
                + 1
                + self.servant_color_powerup[:, None, None, None]     # 从者色卡性能提升
                + (self.servant_color[:, None, None, None, :] * (
                + self.support_color_powerup[None, :, None, None, :]  # 辅助色卡性能提升
                + self.craft_color_powerup  [None, None, :, None, :]  # 礼装色卡性能提升
                + self.mystic_color_powerup [None, None, None, :, :]  # 御主服色卡性能提升
                )).sum(axis=-1)
              )
            * self.servant_class_multiplier[:, None, None, None]  # 从者职阶攻击力补正
            * self.servant_class_relation[:, np.where(CLASS_TYPE == enemy.cls)[0][0]][:, None, None, None]  # 从者敌方职阶克制关系
            * self.servant_attribute_relation[:, np.where(ATTRIBUTE_TYPE == enemy.attribute)[0][0]][:, None, None, None]  # 从者敌方属性 (天地人) 克制关系
            * (
                + 1
                + self.servant_atk_powerup[:, None, None, None]  # 从者攻击力提升
                + self.support_atk_powerup[None, :, None, None]  # 辅助攻击力提升
                + self.craft_atk_powerup  [None, None, :, None]  # 礼装攻击力提升
                + self.mystic_atk_powerup [None, None, None, :]  # 御主服攻击力提升
              )
            * (
                + 1
                + self.servant_noble_powerup[:, None, None, None]  # 从者宝具威力提升
                + self.support_noble_powerup[None, :, None, None]  # 辅助宝具威力提升
                + self.craft_noble_powerup  [None, None, :, None]  # 礼装宝具威力提升
                + self.mystic_noble_powerup [None, None, None, :]  # 御主服宝具威力提升
                + (enemy.trait * self.servant_skill_extra).sum(axis=-1)[:, None, None, None]  # 从者技能特攻
              )
            * (enemy.trait * self.servant_noble_extra + 1).prod(axis=1)[:, None, None, None]  # 从者宝具特攻
            + self.support_atk_addition[None, :, None, None]  # 伤害固定值增加
        )
    
    def calculate_np(self):
        return (
            + self.servant_np[:, None, None, None]
            + self.support_np[None, :, None, None]
            + self.craft_np  [None, None, :, None]
            + self.mystic_np [None, None, None, :]
        )
    
    def output(self, enemy, slic=(slice(None))):
        damage = nob.calculate_damage(enemy)
        mask = (nob.calculate_np() >= 1) * (damage > 100000)
        arg_mask = np.array(np.where(mask[slic])).T
        com = pd.DataFrame(arg_mask, dtype=object)
        com["伤害"] = damage[tuple(arg_mask.T)]
        com.columns = ("从者", "队友", "礼装", "衣服", "伤害")
        for idx in range(len(com)):
            com.at[idx, "从者"] = nob.servant_name[com["从者"][idx]]
            com.at[idx, "队友"] = nob.support_name[com["队友"][idx]]
            com.at[idx, "礼装"] = nob.craft_name[com["礼装"][idx]]
            com.at[idx, "衣服"] = nob.mystic_name[com["衣服"][idx]]
        com    
        com.to_excel('组合结果2.xlsx')
        return com

Noble_All()

## 结果展示

譬如对第一个敌人而言：

In [11]:
df_enemy.iloc[0]

敌人    飞龙
职阶     R
阵营    di
特性     龙
Name: 0, dtype: object

伤害表如下 (未排序) (第一次计算耗时较长，约 1.3 s，后面大约是 320 ms)：

In [14]:
%%time
nob = Noble_All()
enemy = Enemy(df_enemy.iloc[0])
nob.output(enemy)

Wall time: 327 ms


,从者,队友,礼装,衣服,伤害
0,齐格飞,单孔明,醉贞,初始,101574.581919
1,齐格飞,单孔明,相扑,初始,101030.322081
2,齐格飞,单孔明,空骑,初始,105043.634649
3,齐格飞,单梅林,宝石,初始,107459.923430
4,齐格飞,单CBA,醉贞,初始,101074.581919
...,...,...,...,...,...
1104,R蛇神,明梅,空骑,新年,106893.837382
1105,R蛇神,明CBA,黑杯,初始,126440.186299
1106,R蛇神,明CBA,黑杯,换人,115348.941886
1107,R蛇神,明CBA,黑杯,极地,121301.243054


如果对其中的结果进行截取 (单从者拐)，则一共有 113 个 hit 情况：

In [15]:
%%time
nob = Noble_All()
enemy = Enemy(df_enemy.iloc[0])
nob.output(enemy, slic=(slice(0, 51), slice(0, 3), slice(0, 7), slice(0, 7)))

Wall time: 63.8 ms


,从者,队友,礼装,衣服,伤害
0,齐格飞,单孔明,醉贞,初始,101574.581919
1,齐格飞,单孔明,相扑,初始,101030.322081
2,齐格飞,单孔明,空骑,初始,105043.634649
3,齐格飞,单梅林,宝石,初始,107459.923430
4,齐格飞,单CBA,醉贞,初始,101074.581919
...,...,...,...,...,...
108,B喵,单CBA,空骑,初始,114713.984807
109,B喵,单CBA,空骑,换人,102638.828512
110,B喵,单CBA,空骑,泳装,100781.112158
111,B喵,单CBA,空骑,极地,112068.427837
